# Leer registros de pruebas

In [35]:
import pandas as pd

df = pd.read_csv("experimentos_log_expanded.csv")

# Vista rápida
print(df.head())

# Ver cuántos experimentos guardaste
print("Total experimentos:", len(df))

# Ver qué algoritmos probaste
print(df["algoritmo"].unique())

                                           timestamp  script  algoritmo  \
0  2025-09-04T17:39:51,20250901_PruebasEntrenamie...     NaN        NaN   
1  2025-09-04T17:59:07,20250901_PruebasEntrenamie...     NaN        NaN   
2  2025-09-04T18:25:25,20250901_PruebasEntrenamie...     NaN        NaN   
3  2025-09-04T18:25:57,20250901_PruebasEntrenamie...     NaN        NaN   

   dataset  clases_removidas  seed  n_train  n_test  n_features  num_classes  \
0      NaN               NaN   NaN      NaN     NaN         NaN          NaN   
1      NaN               NaN   NaN      NaN     NaN         NaN          6.0   
2      NaN               NaN   NaN      NaN     NaN         NaN          6.0   
3      NaN               NaN   NaN      NaN     NaN         NaN          NaN   

   fit_seconds  pred_seconds  ms_per_sample  OA  F1_macro  
0          NaN           NaN            NaN NaN       NaN  
1          NaN           NaN            NaN NaN       NaN  
2          NaN           NaN            NaN

In [18]:
with open("experimentos_log.csv", encoding="utf-8") as f:
    for i in range(5):
        print(f.readline())


timestamp,script,algoritmo,dataset,clases_removidas,seed,n_train,n_test,n_features,num_classes,fit_seconds,pred_seconds,ms_per_sample,OA,F1_macro,system_info

"2025-09-04T17:39:51,20250901_PruebasEntrenamientoRF.ipynb,RandomForest,PuntosMuestra_CR_2023.csv,2,42,20439,8760,13,6,1.0931,0.0719,0.008,0.8826,0.8161,""{""""host"""": """"IALB18"""", """"user"""": """"Fran"""", """"os"""": """"Windows 10"""", """"machine"""": """"AMD64"""", """"processor"""": """"Intel64 Family 6 Model 183 Stepping 1, GenuineIntel"""", """"python"""": """"3.11.13"""", """"cpu_count_logical"""": 32, """"cpu_count_physical"""": 24, """"ram_gb"""": 127.71, """"numpy"""": """"2.3.2"""", """"pandas"""": """"2.3.2"""", """"sklearn"""": """"1.7.1"""", """"torch"""": """"2.8.0+cpu"""", """"torchvision"""": """"0.23.0+cpu"""", """"timm"""": """"1.0.19"""", """"xgboost"""": """"3.0.4"""", """"cuda_available"""": false, """"cuda_version"""": null, """"gpu_name"""": """"""""}""",,,,,,,,,,,,,,,

"2025-09-04T17:59:07,202509

In [23]:
import csv

SRC = "experimentos_log.csv"
DST = "experimentos_log_fixed.csv"

# Cabecera esperada
COLUMNS = [
    "timestamp","script","algoritmo","dataset","clases_removidas","seed",
    "n_train","n_test","n_features","num_classes",
    "fit_seconds","pred_seconds","ms_per_sample","OA","F1_macro","system_info"
]

with open(SRC, encoding="utf-8") as f:
    lines = f.read().splitlines()

# parseamos la cabecera original si existe; si no, forzamos la correcta
try:
    header = next(csv.reader([lines[0]]))
except Exception:
    header = COLUMNS

if header != COLUMNS:
    header = COLUMNS
    data_lines = lines  # por si tu archivo no tenía cabecera válida
else:
    data_lines = lines[1:]

out_rows = []

for i, line in enumerate(data_lines, start=2):
    fixed_line = line
    # 1) si la fila está envuelta en comillas, quitamos la 1ª y cortamos en la última
    if fixed_line.startswith('"'):
        lastq = fixed_line.rfind('"')
        if lastq > 0:
            fixed_line = fixed_line[1:lastq]  # sin comillas exteriores
    # 2) ahora parseamos la fila ya "desenvuelta"
    row = next(csv.reader([fixed_line]))
    # si sobran comas al final (celdas vacías), las ignoramos
    if len(row) > len(COLUMNS):
        row = row[:len(COLUMNS)]
    # si faltan columnas, completamos con vacíos
    if len(row) < len(COLUMNS):
        row = row + [""] * (len(COLUMNS) - len(row))
    out_rows.append(row)

with open(DST, "w", newline="", encoding="utf-8") as fout:
    w = csv.writer(fout)
    w.writerow(COLUMNS)
    w.writerows(out_rows)

print(f"Reparado → {DST} con {len(out_rows)} filas.")


Reparado → experimentos_log_fixed.csv con 4 filas.


In [ ]:
import pandas as pd, json, re

# lee el CSV reparado que ya te funciona
df = pd.read_csv("experimentos_log_fixed.csv", engine="python")

def fix_sysinfo_cell(s):
    if pd.isna(s): 
        return None
    t = str(s).strip()
    # quitar comillas exteriores si quedaron
    if t.startswith('"') and t.endswith('"'):
        t = t[1:-1]
    # colapsar comillas duplicadas
    t = t.replace('""""', '"').replace('""', '"')
    t = re.sub(r'\s+', ' ', t)
    try:
        obj = json.loads(t)
        # devolver como JSON bien formateado
        return json.dumps(obj, ensure_ascii=False)
    except Exception:
        # si no pudo parsear, lo dejamos vacío
        return None

# normaliza la columna
df["system_info"] = df["system_info"].apply(fix_sysinfo_cell)

# guarda un CSV limpio con el JSON ya normalizado
df.to_csv("experimentos_log_clean.csv", index=False)
print("Guardado: experimentos_log_clean.csv")

# (opcional) expandir system_info -> columnas
sysdict = df["system_info"].dropna().apply(json.loads)
df_sys = pd.json_normalize(sysdict)
df_full = pd.concat([df.drop(columns=["system_info"]), df_sys], axis=1)

df_full.to_csv("experimentos_log_expanded.csv", index=False)
print("Guardado: experimentos_log_expanded.csv")
print(df_full.head())


Guardado: experimentos_log_clean.csv
Guardado: experimentos_log_expanded.csv
                                           timestamp  script  algoritmo  \
0  2025-09-04T17:39:51,20250901_PruebasEntrenamie...     NaN        NaN   
1  2025-09-04T17:59:07,20250901_PruebasEntrenamie...     NaN        NaN   
2  2025-09-04T18:25:25,20250901_PruebasEntrenamie...     NaN        NaN   
3  2025-09-04T18:25:57,20250901_PruebasEntrenamie...     NaN        NaN   

   dataset  clases_removidas  seed  n_train  n_test  n_features  num_classes  \
0      NaN               NaN   NaN      NaN     NaN         NaN          NaN   
1      NaN               NaN   NaN      NaN     NaN         NaN          6.0   
2      NaN               NaN   NaN      NaN     NaN         NaN          6.0   
3      NaN               NaN   NaN      NaN     NaN         NaN          NaN   

   fit_seconds  pred_seconds  ms_per_sample  OA  F1_macro  
0          NaN           NaN            NaN NaN       NaN  
1          NaN           N

In [38]:
import pandas as pd

df = pd.read_csv("experimentos_log_clean.csv")
print(df.columns.tolist())
print(df[["algoritmo","system_info"]].head())
print("Nulos en system_info:", df["system_info"].isna().sum())


['timestamp', 'script', 'algoritmo', 'dataset', 'clases_removidas', 'seed', 'n_train', 'n_test', 'n_features', 'num_classes', 'fit_seconds', 'pred_seconds', 'ms_per_sample', 'OA', 'F1_macro', 'system_info']
               algoritmo  system_info
0           RandomForest          NaN
1            MLP_tabular          NaN
2  ViT_tiny_linear_probe          NaN
3                XGBoost          NaN
Nulos en system_info: 4
